In [4]:
from imapclient import IMAPClient
 
# 邮箱登录
email_address = '2548538192@qq.com'  # 邮箱地址
password = 'yelkviglvagkecca'  # 密码
# 邮箱服务器信息
s_mail = IMAPClient('imap.qq.com')  # HOST = "imap.host.com"
s_mail.login(email_address, password)

b'Success login ok'

In [5]:
s_mail.list_folders()

[((b'\\NoSelect', b'\\HasChildren'), b'/', '其他文件夹'),
 ((b'\\HasNoChildren',), b'/', 'INBOX'),
 ((b'\\HasNoChildren',), b'/', 'Sent Messages'),
 ((b'\\HasNoChildren',), b'/', 'Drafts'),
 ((b'\\HasNoChildren',), b'/', 'Deleted Messages'),
 ((b'\\HasNoChildren',), b'/', 'Junk'),
 ((b'\\HasNoChildren',), b'/', '其他文件夹/QQ邮件订阅')]

In [7]:
s_mail.select_folder('INBOX', readonly=True)

{b'PERMANENTFLAGS': (),
 b'EXISTS': 983,
 b'RECENT': 0,
 b'UNSEEN': [b'614'],
 b'UIDVALIDITY': 1711042127,
 b'UIDNEXT': 1078,
 b'FLAGS': (b'\\Answered', b'\\Flagged', b'\\Deleted', b'\\Draft', b'\\Seen'),
 b'READ-ONLY': [b'']}

In [13]:

import datetime
message_ids = s_mail.search(['SINCE', datetime.date(2024, 3, 24)])
message_ids

[1061,
 1062,
 1063,
 1064,
 1065,
 1066,
 1067,
 1068,
 1069,
 1070,
 1071,
 1072,
 1073,
 1074,
 1075,
 1076,
 1077]

In [12]:
for id in message_ids[:4]:
    print(s_mail.fetch(id, ['BODY[]']))

defaultdict(<class 'dict'>, {462: {b'SEQ': 454, b'BODY[]': b'Received: from mail-oi1-x22b.google.com (mail-oi1-x22b.google.com [2607:f8b0:4864:20::22b])\r\n\tby newxmmxszb1-9.qq.com (NewMX) with SMTP id 87A22A8\r\n\tfor <2548538192@qq.com>; Tue, 19 Mar 2024 13:02:07 +0800\r\nX-QQ-mid: xmmxszb1-9t1710824527t9wuu5op0\r\nSender: info@moththe.com\r\nX-QQ-CSender: info@moththe.com\r\nX-QQ-XMAILINFO: Mk5GybA3Yrmikb5ZtimL35XJVN+yEama5aSofvQpEXrnJOP73HXT6d+M3NwWjF\r\n\t ROpkCwl5ZN6QArBoaVWRifFJdB4XT8lvj1+d8DscIK/BnakckaXcny2umD3PZfF5sfvWSyoZi79N\r\n\t v2A8UIIOGBq8szXlcfRaidv11MVNyNAHZ2k0fHvJrFVeeHGxCiqSHAzD01JfaX3Y8mTYU0bE0NwN\r\n\t +dnK5VmILNpXMq8S6ZNOO5apJ1ekZHHnSz5vUTKhVmmu/6CWz2atvgWcKpxIuDLQ6+w+xWNqyCIS\r\n\t LrZNZhdfpYcn4qfU3dYDiEDi7bKKSLn4foSraMeOw4a3OmKPSNMgTdyUa+JYMOWzjwINQwibpD84\r\n\t YpH8RD2Hr49gqKyCEzcMBQwgTDj7y8Nl8lBSokPUXrlsbJnLo+jBh5/JUqBXylgmIPsnYCmhuTaB\r\n\t /+d0/t0IA8X8wdOOXbvd6mTgyWTSUEKZoV1qKg7Ar5nLu6CmTuzJsFinF85tHyWbwwxmXAv0PkJs\r\n\t JGDYeX1xkFJ9xC3QznpcdfX+lLgOIJAfm0i

In [14]:
import sys
import codecs

for msgid, data in s_mail.fetch(messages, ['ENVELOPE']).items():
    envelope = data[b'ENVELOPE']
    print('ID #%d: "%s" received %s' % (msgid, envelope.subject.decode(), envelope.date))

    response = s_mail.fetch([msgid], ['BODY[]', 'FLAGS'])
    message_bytes = response[msgid][b'BODY[]']
    email_message = email.message_from_bytes(message_bytes)

    for part in email_message.walk():
        if part.get_content_maintype() == "multipart":
            continue
        if part.get('Content-Disposition') is None:
            continue
        fileName = part.get_filename()

        if bool(fileName):
            filePath = os.path.join('/path/to/save/attachments', fileName)
            if not os.path.isfile(filePath):
                print(fileName)
                with codecs.open(filePath, 'wb', encoding='utf-8') as f:
                    f.write(part.get_payload(decode=True).decode('utf-8'))

s_mail.logout()

UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-5: ordinal not in range(128)